In [1]:
key = pd.read_csv('../data/key.csv')
train = pd.read_csv('../data/train.csv')
weather = pd.read_csv('../data/weather.csv')
test = pd.read_csv('../data/test.csv')

In [2]:
def match_dateformat(df, year):
    """
    영문 월을 숫자 월로 바꾸어주고 나중에 사용하기 쉽도록 datetime.date 형태로 바꾸어주는 함수
    """
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i in range(len(df)):
        dates = df.loc[i][0]
        dates = dates.split(" ")
        for j in range(len(months)):
            if dates[0] == months[j]:
                dates[0] = str(j + 1)
                dates_df = ["{} {} {}".format(year, dates[0], dates[1])]
                dates_df = pd.to_datetime(dates_df)
                df.loc[i][0] = dates_df.date[0]
    return df

In [3]:
def merge_holiday(holiday_df1, holiday_df2, holiday_df3):
    """
    각 연도별 공휴일 리스트 합치기
    """
    frame = [holiday_df1, holiday_df2, holiday_df3]
    holiday = pd.concat(frame).reset_index(drop=True)
    return holiday

In [4]:
def find_holiday(file, year):
    """
    수요에 영향을 미치는 주요 공휴일을 찾아내는 함수
    """
    holidays = ["New Year's Day", "Martin Luther King Jr. Day", "Valentine's Day",  "President's Day", "Easter Sunday", 
                      "Mother's Day", "Memorial Day", "Father's Day", "Independence Day", "Labor Day", "Columbus Day",
                      "Halloween", "Veterans Day", "Thanksgiving Day", "Black Friday", "Christmas Eve", "Christmas Day", "New Year's Eve"]
    
    holi = pd.read_excel(file, year, header=None)
    holi = match_dateformat(holi, year)
    holiday = pd.DataFrame(columns=[0,1,2,3,4])
    for _ in holidays:
        for i in range(len(holi[2])):
            if _ == holi[2][i]:
                holiday = holiday.append(holi.loc[i])
    return holiday

In [5]:
def TM_transform(series, T_replace, M_replace): 
    """
    데이터내의 T, M을 원하는 값으로 바꿔주는 함수
    TM_transform(series, T_replace, M_replace)
    """
    series = series.astype(str).map(lambda s: s.strip())
    series[series == 'T'] = T_replace
    series[series =='M'] = M_replace
    
    return series.astype('float')

In [6]:
holiday12 = find_holiday('../data/holiday.xlsx', '2012')
holiday13 = find_holiday('../data/holiday.xlsx', '2013')
holiday14 = find_holiday('../data/holiday.xlsx', '2014')
holiday = merge_holiday(holiday12, holiday13, holiday14)

In [7]:
train_df = train[train['date'] <= '2013-03-31'].reset_index(drop=True)
weather_df = weather[weather['date'] <= '2013-03-31'].reset_index(drop = True)

In [8]:
weather_df = weather_df.drop(['sunrise', 'sunset', 'codesum'], axis = 1)

In [9]:
column_list = weather_df.columns[2:-1]
column_list

Index(['tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir'],
      dtype='object')

In [10]:
for column in column_list :
    weather_df['{}'.format(column)] = TM_transform(weather_df['{}'.format(column)], 0, np.NaN)

In [11]:
weather_df['date'] = pd.to_datetime(weather_df['date'])

In [12]:
weather_df['event'] = ""
for num in range(len(weather_df)) :
    if weather_df.at[num, 'snowfall'] >= 2 or weather_df.at[num, 'preciptotal'] >= 1 :
        weather_df.set_value(num, 'event', True)
    else :
        weather_df.set_value(num, 'event', False)

In [13]:
train_df['log_units'] = np.log(train_df['units'] + 1)

In [14]:
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['weekday'] = train_df.date.dt.weekday  # 월요일이 0 일요일이 6
train_df['weekend'] = train_df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일
train_df['holiday'] = train_df.date.isin([5, 6])

In [15]:
df = pd.merge(train_df, key, on = 'store_nbr')
df = pd.merge(df, weather_df, on = ['date', 'station_nbr'])

In [16]:
df.head()

,date,store_nbr,item_nbr,units,log_units,weekday,weekend,holiday,station_nbr,tmax,...,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,event
0,2012-01-01,1,1,0,0.0,6,True,False,1,52.0,...,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False
1,2012-01-01,1,2,0,0.0,6,True,False,1,52.0,...,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False
2,2012-01-01,1,3,0,0.0,6,True,False,1,52.0,...,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False
3,2012-01-01,1,4,0,0.0,6,True,False,1,52.0,...,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False
4,2012-01-01,1,5,0,0.0,6,True,False,1,52.0,...,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False


In [32]:
tmp_ = df.pivot_table(values = 'log_units', index = ['date', 'store_nbr', 'item_nbr'])
tmp_ = tmp[tmp['log_units'] != 0]

In [33]:
tmp_ = tmp_.reset_index()

In [34]:
holiday.head()

,0,1,2,3,4
0,2012-01-01,Sunday,New Year's Day,Federal Holiday,NaN
1,2012-01-16,Monday,Martin Luther King Jr. Day,Federal Holiday,NaN
2,2012-02-14,Tuesday,Valentine's Day,Observance,NaN
3,2012-04-08,Sunday,Easter Sunday,"Observance, Christian",NaN
4,2012-05-13,Sunday,Mother's Day,Observance,NaN


In [35]:
tmp_.head() #weekday, holiday, event 추가해야함

,date,store_nbr,item_nbr,log_units
0,2012-01-01,1,9,3.401197
1,2012-01-01,1,28,1.098612
2,2012-01-01,1,51,0.693147
3,2012-01-01,2,5,5.257495
4,2012-01-01,2,44,5.375278


In [36]:
weather_df.head()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,event
0,1,2012-01-01,52.0,31.0,42.0,NaN,36.0,40.0,23.0,0.0,NaN,0.05,29.78,29.92,3.6,20.0,4.6,False
1,2,2012-01-01,48.0,33.0,41.0,16.0,37.0,39.0,24.0,0.0,0.0,0.07,28.82,29.91,9.1,23.0,11.3,False
2,3,2012-01-01,55.0,34.0,45.0,9.0,24.0,36.0,20.0,0.0,0.0,0.00,29.77,30.47,9.9,31.0,10.0,False
3,4,2012-01-01,63.0,47.0,55.0,4.0,28.0,43.0,10.0,0.0,0.0,0.00,29.79,30.48,8.0,35.0,8.2,False
4,6,2012-01-01,63.0,34.0,49.0,0.0,31.0,43.0,16.0,0.0,0.0,0.00,29.95,30.47,14.0,36.0,13.8,False


In [37]:
tmp_ = pd.merge(tmp_, key, on = 'store_nbr')
tmp_['date'] = pd.to_datetime(tmp_['date'])
tmp_['weekday'] = tmp_.date.dt.weekday  # 월요일이 0 일요일이 6
tmp_['weekend'] = tmp_.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일
tmp_['holiday'] = tmp_.date.isin([5, 6])

In [38]:
tmp_test = pd.merge(tmp_, weather_df[['date', 'station_nbr', 'event']], on = ['date', 'station_nbr'])

In [39]:
tmp_test.head()

,date,store_nbr,item_nbr,log_units,station_nbr,weekday,weekend,holiday,event
0,2012-01-01,1,9,3.401197,1,6,True,False,False
1,2012-01-01,1,28,1.098612,1,6,True,False,False
2,2012-01-01,1,51,0.693147,1,6,True,False,False
3,2012-01-02,1,9,4.110874,1,0,False,False,False
4,2012-01-02,1,28,1.791759,1,0,False,False,False


In [44]:
model = sm.OLS.from_formula("log_units ~ C(store_nbr) + C(item_nbr) + C(weekday) + C(holiday) + C(event) + 0", data = tmp_test)

In [45]:
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     2841.
Date:                Sat, 03 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:51:26   Log-Likelihood:                -45481.
No. Observations:               61318   AIC:                         9.126e+04
Df Residuals:                   61169   BIC:                         9.260e+04
Df Model:                         148                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(store_nbr)[1]        1.2829      0

C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [ ]:
sp.stats.probplot(result.resid, plot=plt)
plt.show()

In [29]:
fig = sm.graphics.plot_regress_exog(result, "event")
fig.suptitle("")
plt.show()

ValueError: 'event' is not in list